In [2]:
import mlflow
import mlflow.tensorflow
import tensorflow as tf
from pyngrok import ngrok
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
data_directory = r'G:\Meu Drive\reconhecimento_caes\dados_organizados'

In [4]:
train_set = tf.keras.utils.image_dataset_from_directory(
    directory = data_directory,
    labels='inferred',
    validation_split = 0.2,
    subset = "training",
    seed=123,
    image_size = (224, 224),
    batch_size = 32,

)

validation_set = tf.keras.utils.image_dataset_from_directory(
    directory = data_directory,
    labels='inferred',
    validation_split = 0.2,
    subset = "validation",
    seed=123,
    image_size = (224, 224),
    batch_size = 32
)

Found 115 files belonging to 25 classes.
Using 92 files for training.
Found 115 files belonging to 25 classes.
Using 23 files for validation.


In [5]:
def preprocessing_pipeline(image: tf.Tensor, label: tf.Tensor):
  resized_image = tf.image.resize(image, [224, 224])
  normalized_image = tf.keras.applications.resnet50.preprocess_input(resized_image)

  return normalized_image, label

In [6]:
train_dataset = train_set.map(preprocessing_pipeline)
validation_dataset = validation_set.map(preprocessing_pipeline)

In [7]:
base_model = tf.keras.applications.ResNet50(
    include_top = False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)

base_model.trainable = False

In [8]:
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(units=25, activation='softmax')
])

In [9]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [10]:
mlflow.set_experiment("Artigo - Reconhecimento de Cães")
mlflow.tensorflow.autolog()

with mlflow.start_run(run_name="Baseline de Classificação com Resnet50"):
    history = model.fit(
        train_dataset,
        epochs=10,
        validation_data=validation_dataset
    )

2025/07/14 21:34:18 INFO mlflow.tracking.fluent: Experiment with name 'Artigo - Reconhecimento de Cães' does not exist. Creating a new experiment.


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0353 - loss: 4.1607

3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.0374 - loss: 4.1444 - val_accuracy: 0.0870 - val_loss: 3.3058
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1590 - loss: 3.0021

3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.1681 - loss: 2.9842 - val_accuracy: 0.2609 - val_loss: 2.9815
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4683 - loss: 1.9904

3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.4681 - loss: 1.9858 - val_accuracy: 0.3043 - val_loss: 2.5044
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6148 - loss: 1.4138

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.9416 - loss: 0.4349 - val_accuracy: 0.5652 - val_loss: 1.4855
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9787 - loss: 0.2541

3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.9759 - loss: 0.2532 - val_accuracy: 0.5652 - val_loss: 1.4240
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.1894

3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 1.0000 - loss: 0.1898 - val_accuracy: 0.6522 - val_loss: 1.3775
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.1317

3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 1.0000 - loss: 0.1285 - val_accuracy: 0.6957 - val_loss: 1.3411
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


2025/07/14 21:35:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


In [12]:
print("Avaliando o modelo diretamente no conjunto de validação...")
results = model.evaluate(validation_dataset)
print("\nResultados da Avaliação:")
print(f"Perda de Validação (Loss): {results[0]}")
print(f"Acurácia de Validação (Accuracy): {results[1]}")

Avaliando o modelo diretamente no conjunto de validação...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6957 - loss: 1.3411

Resultados da Avaliação:
Perda de Validação (Loss): 1.3410615921020508
Acurácia de Validação (Accuracy): 0.695652186870575
